In [1]:
import numpy as np
import pandas as pd
import itertools
import re
import string


In [2]:
real_df = pd.read_csv('./True.csv')
fake_df = pd.read_csv('./Fake.csv')

In [3]:
fake_df['label'] = ("FAKE")
real_df['label'] = ("TRUE")

In [4]:
df = pd.concat([fake_df, real_df], ignore_index=True, sort=False)
df.head()

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",FAKE
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",FAKE
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",FAKE
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",FAKE
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",FAKE


In [5]:
df.tail()

,title,text,subject,date,label
44893,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017",TRUE
44894,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017",TRUE
44895,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017",TRUE
44896,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017",TRUE
44897,Indonesia to buy $1.14 billion worth of Russia...,JAKARTA (Reuters) - Indonesia will buy 11 Sukh...,worldnews,"August 22, 2017",TRUE


In [6]:
sentences = df['text'].astype(str).str.lower().values.tolist()
labels = df['label'].values.tolist()

In [7]:
len(sentences)

44898

In [8]:
df.isnull().sum()

title      0
text       0
subject    0
date       0
label      0
dtype: int64

In [9]:
 labels = df.label

In [10]:
labels.head()

0    FAKE
1    FAKE
2    FAKE
3    FAKE
4    FAKE
Name: label, dtype: object

In [11]:
labels.tail()

44893    TRUE
44894    TRUE
44895    TRUE
44896    TRUE
44897    TRUE
Name: label, dtype: object

In [12]:
labels

0        FAKE
1        FAKE
2        FAKE
3        FAKE
4        FAKE
         ... 
44893    TRUE
44894    TRUE
44895    TRUE
44896    TRUE
44897    TRUE
Name: label, Length: 44898, dtype: object

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
x_train, x_test, y_train, y_test = train_test_split(df["text"], labels, test_size=0.2, random_state=20)

In [15]:
x_train.head()

43673    BERLIN (Reuters) - Germany deported eight Afgh...
32959    WASHINGTON (Reuters) - The U.S. House of Repre...
13191    The video below is a great snapshot of what wa...
25778    WASHINGTON (Reuters) - The U.S. State Departme...
9796     The owner of the Ringling Bar, located south o...
Name: text, dtype: object

In [16]:
x_test

10506    The Windy City is under fire for turning publi...
23290    Tune in to the Alternate Current Radio Network...
39424    YANGON (Reuters) - Myanmar insisted on Wednesd...
4774     Donald Trump, knowing that his poll numbers wi...
31906    NEW YORK (Reuters) - An artists’ collective to...
                               ...                        
18607    Tick tock. America will now wait until conserv...
13242    Communities like Ferguson, Baltimore and now M...
30511    WASHINGTON (Reuters) - A Republican antitrust ...
36135    DUBAI (Reuters) - British Foreign Secretary Bo...
9759     Late-night TV host Jimmy Kimmel opened his pro...
Name: text, Length: 8980, dtype: object

In [17]:
y_train

43673    TRUE
32959    TRUE
13191    FAKE
25778    TRUE
9796     FAKE
         ... 
31962    TRUE
23452    FAKE
23775    TRUE
37135    TRUE
27098    TRUE
Name: label, Length: 35918, dtype: object

In [18]:
y_test

10506    FAKE
23290    FAKE
39424    TRUE
4774     FAKE
31906    TRUE
         ... 
18607    FAKE
13242    FAKE
30511    TRUE
36135    TRUE
9759     FAKE
Name: label, Length: 8980, dtype: object

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier

In [20]:
# intalize a tf idf vectorizer
vector = TfidfVectorizer(stop_words='english' , max_df=0.7)

In [21]:
#fit and transform

tf_train = vector.fit_transform(x_train)
tf_test = vector.transform(x_test)


In [22]:
# intalize a passive aggresive classifier
pac = PassiveAggressiveClassifier(max_iter=50)
pac.fit(tf_train, y_train)

PassiveAggressiveClassifier(max_iter=50)

In [23]:
# prediction the test dataset
from sklearn.metrics import accuracy_score, confusion_matrix
y_pred = pac.predict(tf_test)

In [24]:
score = accuracy_score(y_test, y_pred)

In [25]:
print(f"Accuracy:{round(score*100,2)}%")

Accuracy:99.5%


In [26]:
#confusion metrices
confusion_matrix(y_test, y_pred, labels=['FAKE', 'REAL'])

array([[4677,    0],
       [   0,    0]], dtype=int64)

In [27]:
#save model
import pickle
with open('finalized_pkl', 'wb') as f:
    pickle.dump(pac, f)